# Imports

In [1]:
!pip install mercantile geopandas rasterio pillow requests leafmap
!pip install "ultralytics<=11.3.40"
!pip install roboflow
!pip install inference
!pip install supervision

In [8]:
import geopandas as gpd
import mercantile
import requests
from rasterio.transform import from_bounds
from rasterio.enums import Resampling
from PIL import Image
import io
import os
import json
import glob
import leafmap
from datetime import datetime
import supervision as sv
import cv2

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Interactive Map

In [16]:
from google.colab import output
output.enable_custom_widget_manager()

m = leafmap.Map(center=[41.36, 2.15], zoom=15, height="500px")
m.add_basemap("SATELLITE")
m

Map(center=[41.36, 2.15], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [17]:
# Get the drawn features as GeoJSON
interactive_geojson_data = m.draw_features

bboxes = []

print(interactive_geojson_data)
# Extract bounding boxes from the GeoJSON
for feature in interactive_geojson_data:
    if feature['geometry']['type'] == 'Polygon':
        polygon_coords = feature['geometry']['coordinates'][0]
        # Calculate bounding box (min_lon, min_lat, max_lon, max_lat)
        min_lon = min(coord[0] for coord in polygon_coords)
        min_lat = min(coord[1] for coord in polygon_coords)
        max_lon = max(coord[0] for coord in polygon_coords)
        max_lat = max(coord[1] for coord in polygon_coords)
        bbox = [min_lon, min_lat, max_lon, max_lat]
        bboxes.append(bbox)
        print(f"Bounding box for polygon: {bbox}")# Get the drawn features as GeoJSON

[{'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[2.106543, 41.387209], [2.106543, 41.395644], [2.127528, 41.395644], [2.127528, 41.387209], [2.106543, 41.387209]]]}}]
Bounding box for polygon: [2.106543, 41.387209, 2.127528, 41.395644]


# Constants

In [19]:
# viewer here: https://www.arcgis.com/apps/mapviewer/index.html
TILE_URL = "https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
OUTPUT_FILE = "/content/drive/MyDrive/Colab Notebooks/Satellite Imagery Download/From Afar 26/Barcelona Pools"

# Utility functions

In [23]:
def get_tiles_for_bbox(bounds, zoom):
    min_lon, min_lat, max_lon, max_lat = bounds
    tiles = list(mercantile.tiles(min_lon, min_lat, max_lon, max_lat, zoom))
    return tiles

def download_tile(tile):
    url = TILE_URL.format(x=tile.x, y=tile.y, z=tile.z)
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        return Image.open(io.BytesIO(response.content))

    print(f"Failed to download tile {tile.x}, {tile.y}, {tile.z}")
    return None

def stitch_tiles(tiles):
    stitched = Image.new("RGB", (512, 512))
    for i, tile in enumerate(tiles):
        if tile:
            x, y = (i % 2) * 256, (i // 2) * 256
            stitched.paste(tile, (x, y))
    return stitched

def get_current_timestamp():
  now = datetime.now()
  formatted_date = now.strftime("%Y-%m-%d_%H:%M")
  return formatted_date

def create_folder_if_not_exists(full_path):
  """Creates a folder in Google Drive if it doesn't exist.

  Args:
    full_path: The absolute path to the folder to create.
  """
  # drive.mount('/content/drive')  # Mount Google Drive
  # full_path = os.path.join('/content/drive/My Drive', folder_path)

  if not os.path.exists(full_path):
    os.makedirs(full_path)
    print(f"Folder '{full_path}' created in your Google Drive.")
  else:
    print(f"Folder '{full_path}' already exists in your Google Drive.")

# Importing shapefiles or features from the interactive map

In [24]:
# Load bounding box geometries

# USE ONLY ONE OR THE OTHER OF THE TWO OPTIONS:
# You'll need to comment out the one you're not using
# Option 1: Using an existing geojson file exported from QGIS
# It may contain many polygons
# Here is a geojson/shapefile that shows where our bounding boxes should be
# TODO: CHANGE ME with the value of the path from your uploaded geojson
# SHAPEFILE_PATH = "{}/polygons.geojson".format(OUTPUT_FILE)
# gdf = gpd.read_file(SHAPEFILE_PATH)  # Or .shp file
# gdf = gdf.to_crs(epsg=4326)  # Ensure it's in lat/lon format

# Option 2: Using the geojson from the interactive map above
gdf = gpd.GeoDataFrame.from_features(interactive_geojson_data)

# Get their tile coordinates
zoom_level = 18  # CHANGE ME Adjust as needed
gdf["tiles"] = gdf["geometry"].apply(lambda geom: get_tiles_for_bbox(geom.bounds, zoom_level))
for _, shp in gdf.iterrows():
  print("{} images to download for layer at zoom level: {}".format(len(shp["tiles"]), zoom_level))

153 images to download for layer at zoom level: 18


In [25]:
from shapely.geometry import box

metadata_list = []

TILES_FOLDER = "{}/tiles".format(OUTPUT_FILE)
create_folder_if_not_exists(TILES_FOLDER)

for _, row in gdf.iterrows():
    for tile in row["tiles"]:
        subtiles = [
            download_tile(mercantile.Tile(tile.x, tile.y, tile.z)),
            download_tile(mercantile.Tile(tile.x + 1, tile.y, tile.z)),
            download_tile(mercantile.Tile(tile.x, tile.y + 1, tile.z)),
            download_tile(mercantile.Tile(tile.x + 1, tile.y + 1, tile.z))
        ]

        stitched = stitch_tiles(subtiles)
        filename = f"{tile.x}_{tile.y}_{tile.z}.jpg"
        stitched.save(os.path.join(TILES_FOLDER, filename), "JPEG")

        # Get bounding box of tile
        tile_bounds = mercantile.bounds(tile)
        tile_geom = box(tile_bounds.west, tile_bounds.south, tile_bounds.east, tile_bounds.north)

        # Store metadata as GeoJSON feature
        metadata_list.append({
            "filename": filename,
            "geometry": tile_geom
        })

# Convert metadata list to GeoDataFrame
metadata_gdf = gpd.GeoDataFrame(metadata_list, crs="EPSG:4326")

# Save as GeoJSON
metadata_gdf.to_file(os.path.join(TILES_FOLDER, "tile_metadata.geojson"), driver="GeoJSON")

print(f"Saved metadata to {os.path.join(TILES_FOLDER, 'tile_metadata.geojson')}")

Folder '/content/drive/MyDrive/Colab Notebooks/Satellite Imagery Download/From Afar 26/Barcelona Pools/tiles' created in your Google Drive.
Saved metadata to /content/drive/MyDrive/Colab Notebooks/Satellite Imagery Download/From Afar 26/Barcelona Pools/tiles/tile_metadata.geojson


### Object Detection w/ Roboflow

In [6]:
from roboflow import Roboflow
from inference import get_model
# TODO Change this with your Roboflow API key
ROBOFLOW_API_KEY = "nBoGRTuF2O97kIcrOzEX"
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
print(rf.workspace())
project = rf.project("iaac-o4ty0/swimming-pools-dctlb-7yzol")
# project = rf.project("iaac-o4ty0/swimming-pools-for-workshop")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
loading Roboflow workspace...
{
  "name": "IAAC",
  "url": "iaac-o4ty0",
  "projects": [
    "iaac-o4ty0/swimming-pools-dctlb-7yzol",
    "iaac-o4ty0/swimming-pools-for-workshop",
    "iaac-o4ty0/swimmingpools-la",
    "iaac-o4ty0/venice-biennale"
  ]
}


In [7]:
model = get_model(model_id="swimming-pools-for-workshop/1", api_key=ROBOFLOW_API_KEY)

Resolved model_id: swimming-pools-for-workshop/1, dataset_id: swimming-pools-for-workshop, version_id: 1


In [14]:
glob.glob(OUTPUT_FILE)

['/content/drive/MyDrive/Colab Notebooks/Satellite Imagery Download/From Afar 26']

In [29]:
TEST_IMG_FOLDER = "{}/".format(TILES_FOLDER)
DETECTIONS_FOLDER = "{}/detections/".format(OUTPUT_FILE)

create_folder_if_not_exists(DETECTIONS_FOLDER)

for filename in glob.glob(TEST_IMG_FOLDER + "*.jpg"):
  absolute_path = os.path.abspath(filename)
  containing_folder = os.path.dirname(absolute_path)
  fname = os.path.basename(absolute_path)

  # print(filename)
  image = cv2.imread(filename)

  # run inference on our chosen image, image can be a url, a numpy array, a PIL image, etc.
  results = model.infer(image, confidence=.05)[0]
  # print(results)

  # load the results into the supervision Detections api
  detections = sv.Detections.from_inference(results)

  # create supervision annotators
  bounding_box_annotator = sv.BoxAnnotator()
  label_annotator = sv.LabelAnnotator()

  # annotate the image with our inference results
  annotated_image = bounding_box_annotator.annotate(
      scene=image, detections=detections)
  annotated_image = label_annotator.annotate(
      scene=annotated_image, detections=detections)

  # display the image
  # sv.plot_image(annotated_image, size=[4, 4])
  cv2.imwrite("{}/{}".format(DETECTIONS_FOLDER, fname), annotated_image)

Folder '/content/drive/MyDrive/Colab Notebooks/Satellite Imagery Download/From Afar 26/Barcelona Pools/detections/' created in your Google Drive.
/content/drive/MyDrive/Colab Notebooks/Satellite Imagery Download/From Afar 26/Barcelona Pools/tiles/132605_97901_18.jpg
visualization=None inference_id=None frame_id=None time=None image=InferenceResponseImage(width=512, height=512) predictions=[InstanceSegmentationPrediction(x=162.5, y=123.0, width=21.0, height=26.0, confidence=0.0959281325340271, class_name='swimming_pool', class_id=1, detection_id='2f00c1ab-43ee-48b7-9c62-e3a192509b68', parent_id=None, class_confidence=None, points=[Point(x=152.0, y=111.20000457763672), Point(x=152.0, y=135.1999969482422), Point(x=156.0, y=135.1999969482422), Point(x=156.8000030517578, y=134.40000915527344), Point(x=166.40000915527344, y=134.40000915527344), Point(x=167.1999969482422, y=133.60000610351562), Point(x=172.0, y=133.60000610351562), Point(x=172.8000030517578, y=132.8000030517578), Point(x=172.